In [1]:
from ptls.data_load.datasets import ParquetDataset, ParquetFiles

In [2]:
import os


import pandas as pd
import numpy as np
import torch
from functools import partial
import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")

from torch.utils.data import DataLoader

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset

from tqdm.auto import tqdm
import lightgbm as ltb

In [3]:
processed_train = pd.DataFrame(ParquetDataset(ParquetFiles('./Data/train_trx_preprocessor.parquet'))) 
processed_test = pd.DataFrame(ParquetDataset(ParquetFiles('./Data/test_trx_preprocessor.parquet')))

In [4]:
# processed = pd.DataFrame(ParquetDataset(ParquetFiles('./Data/test_tpreprocessor.parquet'))) #загружаем препросцессинг данных train+test

In [5]:
train = MemoryMapDataset(
    data=processed_train.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=22),
        ISeqLenLimit(max_seq_len=2530),
        ToTorch()
    ]
)

test = MemoryMapDataset(
    data=processed_test.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=22),
        ISeqLenLimit(max_seq_len=2530),
        ToTorch()
    ]
)

In [6]:
train_ds = ColesIterableDataset(
    data=train,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

valid_ds = ColesIterableDataset(
    data=test,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

In [7]:
train_dl = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=3,
    train_batch_size=64,
    valid_data=valid_ds,
    valid_num_workers=3,
    valid_batch_size=64
)


In [8]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount': 'log'},
    embeddings={
        "event_type": {'in': 55, "out": 10},
        "event_subtype": {'in': 57, "out": 10},
        'src_type11': {'in': 15, 'out': 1},
        'src_type12': {'in': 78, 'out': 10},
        'dst_type11': {'in': 344, 'out': 30},
        'dst_type12': {'in': 80, 'out': 10},
        'src_type22': {'in': 32845, 'out': 3000},
        'src_type31': {'in': 87, 'out': 10},
        'src_type32': {'in': 2275, 'out': 200},
      }
)

In [9]:
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=32,
    type='gru',
)

In [10]:
model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9)
)

In [11]:
# # Укажите путь к файлу с весами модели
# model_weights_path = 'mlm-emb_trx_v55.pt'

# # Загрузите веса модели
# model.load_state_dict(torch.load(model_weights_path))

# # # Загрузите сохраненное состояние параметров модели

# # model_state_dict = torch.load("model_new1.pt")

# # # Загрузите параметры в модель
# # model.load_state_dict(model_state_dict)


In [12]:
trainer = pl.Trainer(
    max_epochs=10,
    limit_val_batches=5000,
  
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    logger=pl.loggers.TensorBoardLogger(
        save_dir='./logdir',
        name='baseline_result'
    ),
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=5000, save_top_k=-1),
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, train_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 99.3 M
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
99.3 M    Trainable params
0         Non-trainable params
99.3 M    Total params
397.289   Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [14]:
torch.save(model.state_dict(), './mlm-emb_trx_vX1.pt')

In [16]:
import torch
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
from ptls.data_load.datasets import inference_data_loader

In [17]:
processed_train = pd.DataFrame(ParquetDataset(ParquetFiles('./Data/trx_preprocessor.parquet'))) #загружаем препросцессинг данных train+test
id_client = pd.DataFrame(ParquetDataset(ParquetFiles('./Data/result_sample_Client_Month_df_12_06_2024.parquet'))) #загружаем перечень id_client и даты отчётов на которые надо сделаит предсказание

In [18]:
processed_train.shape, id_client.shape

((1081371, 15), (747847, 3))

In [23]:
# Функция для фильтemb_trxрации данных по месяцу
def filter_by_month(timestamps, data_list, month):
    target_date = month - relativedelta(months=1)
    target_year = target_date.year
    target_month = target_date.month
    filtered_timestamps = []
    filtered_data_list = [[] for _ in range(len(data_list))]

    for i, ts in enumerate(timestamps):
        dt = datetime.fromtimestamp(ts)
        
        if dt.year == target_year and dt.month == target_month:
            filtered_timestamps.append(ts)
            for j in range(len(data_list)):
                filtered_data_list[j].append(data_list[j][i])

    filtered_timestamps = torch.tensor(filtered_timestamps)
    filtered_data_list = [torch.tensor(data) for data in filtered_data_list]

    return filtered_timestamps, filtered_data_list

# Основная функция для фильтрации DataFrame по месяцу
def filter_dataframe_by_month(df, timestamp_column, data_columns, month):
    for index, row in tqdm(df.iterrows()):
        timestamps = row[timestamp_column]
        data_list = [row[col] for col in data_columns]
        filtered_timestamps, filtered_data_list = filter_by_month(timestamps, data_list, month)
        
        df.at[index, timestamp_column] = filtered_timestamps
        for col, filtered_data in zip(data_columns, filtered_data_list):
            df.at[index, col] = filtered_data

    return df

# Определяем столбцы с временными метками и данными
timestamp_column = 'event_time'

data_columns = [ 'amount',
 'event_type',
 'event_subtype',
 'currency',
 'src_type11',
 'src_type12',
 'dst_type11',
 'dst_type12',
 'src_type21',
 'src_type22',
 'src_type31',
 'src_type32']



In [24]:
id_client.report_next_end.unique()[0]

Timestamp('2022-11-30 00:00:00')

In [26]:
#фильтруем данные по заданным id_client
df = pd.DataFrame()
for mon in tqdm(id_client.report_next_end.unique()):
    month_to_filter = mon
    date = processed_train[processed_train['client_id'].isin(id_client[id_client['report_next_end'] == mon].client_id)]
    date['report_next_end'] = mon 
    date = filter_dataframe_by_month(date, timestamp_column, data_columns, month_to_filter)
    date = date.applymap(lambda x: np.nan if isinstance(x, list) and len(x) == 0 else x)
    date.dropna(inplace=True)
    df = pd.concat([df,date])

  0%|          | 0/13 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [27]:
# import gc
# del processed_train
# del id_client
# del date[i for i in train]
# amount
# gc.collect()

In [28]:
# Функция для проверки наличия tensor([])
def contains_empty_tensor(x):
    return isinstance(x, torch.Tensor) and x.numel() == 0
# Преобразуем tensor([]) в NaN
df = df.applymap(lambda x: None if contains_empty_tensor(x) else x)

# Удаляем строки, содержащие NaN
df = df.dropna()
df.shape    

(478077, 16)

In [29]:
test = MemoryMapDataset(
    data=df.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id',]),
        ISeqLenLimit(max_seq_len=2530),
        ToTorch(),
    ]
)


In [30]:
trainer = pl.Trainer( max_epochs=-1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [31]:
model.to('cpu')
train_dl = inference_data_loader(test, num_workers=0, batch_size=64)
train_embeds = torch.vstack(trainer.predict(model, train_dl))
train_embeds.shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

torch.Size([478077, 32])

In [32]:
train_df = pd.DataFrame()
train_df['client_id'] = df['client_id']
train_df['report_next_end'] = df['report_next_end']

In [33]:
train_df['emb_trx'] = [i for i in train_embeds.numpy()]
train_df

,client_id,report_next_end,emb_trx
19,00fe1344ab467920aeae2008c2e351d6ecc6d09c0192ae...,2022-11-30,"[1.0, -0.21202722, 0.31116307, 1.0, 1.0, 0.189..."
30,0163fc28a8ec289d03340d56ea13e975b4cf4d0cea6141...,2022-11-30,"[-1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -0.043..."
47,0218864e5f62e44ae86c490de5c3c0822ec3a79ae11476...,2022-11-30,"[-0.0026537764, -0.21202722, 0.31116307, 1.0, ..."
54,02745013e5cf8b2e1896c4f77e89bc3ca952057dd46486...,2022-11-30,"[1.0, -0.21202722, -1.0, -1.0, -0.03291097, 0...."
77,03c4d714e95fcdbfa6b77acf234011d23f51b639a1b697...,2022-11-30,"[-0.0026537764, -0.21202722, -1.0, 1.0, -0.032..."
...,...,...,...
1080734,e26f303b215d861753db9598bdc31022122406469ac3bc...,2022-12-30,"[1.0, -0.21202722, 1.0, 1.0, 0.9999963, -1.0, ..."
1080916,eb7f7ce03df433da1ba312a359d9ec1e071eb966cc6aea...,2022-12-30,"[1.0, 0.9999888, -1.0, 1.0, 1.0, 0.18988582, 1..."
1080928,ebb38c0ca39d3a5aefe95e166704eff9c59f7dcb7c3f5c...,2022-12-30,"[1.0, 1.0, -1.0, 0.9999967, -0.03291097, 0.998..."
1081073,f29acdd92ea6054e8e2ea19eb761e30ff04135aff45203...,2022-12-30,"[-0.0026537764, -0.21202722, -1.0, -0.1718208,..."


In [35]:
train_df.to_parquet(f'./Data/trx_emb_select_1_v2.parquet', engine='pyarrow')